# DCGAN 방법을 사용하여 MNIST 이미지 생성

#### 코드 출처 :  https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py

## 1. 패키지 로드

In [1]:
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

Using TensorFlow backend.
C:\Users\PC\AppData\Local\conda\conda\envs\practice_env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\PC\AppData\Local\conda\conda\envs\practice_env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\PC\AppData\Local\conda\conda\envs\practice_env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np

## 2. MNIST DCGAN Class 생성

In [2]:

class DCGAN():
    
    # Class 초기화 
    def __init__(self):
        # Input shape 
        #MNIST 자료는 28 by 28 by 1 의 크기로 구성 됨 .
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        # 100개의 난수로 이미지를 생성하고자 함
        self.latent_dim = 100
        
        # Optimizer로 Adam을 설정
        optimizer = Adam(0.0002, 0.5)

        # Discriminator 생성
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Generator 생성
        self.generator = self.build_generator()

        # 난수로부터 이미지를 생성
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # Discriminator의 학습을 중지 옵션
        self.discriminator.trainable = False

        # Discriminator가 generator로 부터 생성된 이미지를 input으로 하게끔 설정
        valid = self.discriminator(img)

        # Generator와 Discriminator를 결합 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    # Generator 구조 생성
    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    # Discriminator 구조 생성
    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    # DCGAN 모델 학습
    def train(self, epochs, batch_size=128, save_interval=50):

        # 데이터 로드
        (X_train, _), (_, _) = mnist.load_data()

        # -1 ~ 1 Normalization
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Label 생성
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        #학습 진행
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # 랜덤하게 배치사이즈 만큼 이미지 선택
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Random Normal로 난수 생성 및 이미지 생성
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            #  Discriminator 학습을 위해 진짜 이미지는 1로 라벨
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            
            # 가짜이미지는 0으로 라벨 설정
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            
            # Discriminator loss 설정
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Generator를 학습
            # 가짜이미지를 1로 label 설정
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    #이미지 저장 함수
    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        
        self.createFolder()
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

    #이미지 저장 폴더 생성 함수            
    def createFolder(self):
        if not os.path.exists("./images"):
            os.makedirs("./images")
                



## 3. 실행

In [3]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1

C:\Users\PC\AppData\Local\conda\conda\envs\r-tensorflow\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.914794, acc.: 46.88%] [G loss: 0.765295]
1 [D loss: 0.516690, acc.: 73.44%] [G loss: 1.465075]
2 [D loss: 0.259207, acc.: 89.06%] [G loss: 2.120085]
3 [D loss: 0.240550, acc.: 90.62%] [G loss: 2.759167]
4 [D loss: 0.173096, acc.: 95.31%] [G loss: 3.220384]
5 [D loss: 0.173781, acc.: 95.31%] [G loss: 2.978721]
6 [D loss: 0.222313, acc.: 95.31%] [G loss: 2.910175]
7 [D loss: 0.125392, acc.: 96.88%] [G loss: 2.397475]
8 [D loss: 0.110885, acc.: 98.44%] [G loss: 2.003881]
9 [D loss: 0.129052, acc.: 98.44%] [G loss: 1.991194]
10 [D loss: 0.201300, acc.: 93.75%] [G loss: 1.878527]
11 [D loss: 0.162103, acc.: 100.00%] [G loss: 2.711829]
12 [D loss: 0.574138, acc.: 70.31%] [G loss: 3.435377]
13 [D loss: 0.678097, acc.: 56.25%] [G loss: 3.557718]
14 [D loss: 0.747591, acc.: 54.69%] [G loss: 1.921352]
15 [D loss: 0.368681, acc.: 82.81%] [G loss: 1.728378]
16 [D loss: 0.481489, acc.: 73.44%] [G loss: 0.754716]
17 [D loss: 0.223665, acc.: 90.62%] [G loss: 0.459560]
18 [D loss: 0.10303

151 [D loss: 0.710146, acc.: 57.81%] [G loss: 1.007352]
152 [D loss: 0.984236, acc.: 37.50%] [G loss: 1.145481]
153 [D loss: 0.846087, acc.: 43.75%] [G loss: 0.983927]
154 [D loss: 0.707523, acc.: 54.69%] [G loss: 1.191960]
155 [D loss: 0.839066, acc.: 46.88%] [G loss: 1.240158]
156 [D loss: 0.819283, acc.: 45.31%] [G loss: 0.936007]
157 [D loss: 0.967407, acc.: 39.06%] [G loss: 1.090937]
158 [D loss: 1.001075, acc.: 31.25%] [G loss: 1.254040]
159 [D loss: 0.933499, acc.: 37.50%] [G loss: 1.226724]
160 [D loss: 0.839479, acc.: 48.44%] [G loss: 1.227458]
161 [D loss: 0.905860, acc.: 45.31%] [G loss: 1.296934]
162 [D loss: 0.863898, acc.: 43.75%] [G loss: 1.065543]
163 [D loss: 0.886328, acc.: 43.75%] [G loss: 1.143904]
164 [D loss: 0.808258, acc.: 50.00%] [G loss: 1.137838]
165 [D loss: 0.792929, acc.: 51.56%] [G loss: 1.070440]
166 [D loss: 0.903954, acc.: 45.31%] [G loss: 1.103002]
167 [D loss: 0.804343, acc.: 45.31%] [G loss: 0.893223]
168 [D loss: 0.905776, acc.: 54.69%] [G loss: 0.

301 [D loss: 0.816287, acc.: 46.88%] [G loss: 0.911210]
302 [D loss: 0.761818, acc.: 54.69%] [G loss: 1.020821]
303 [D loss: 0.878055, acc.: 34.38%] [G loss: 1.065889]
304 [D loss: 0.732903, acc.: 50.00%] [G loss: 0.933629]
305 [D loss: 0.787882, acc.: 45.31%] [G loss: 1.011887]
306 [D loss: 0.912727, acc.: 37.50%] [G loss: 1.161441]
307 [D loss: 0.810066, acc.: 50.00%] [G loss: 1.296383]
308 [D loss: 0.822627, acc.: 51.56%] [G loss: 0.927954]
309 [D loss: 0.705696, acc.: 59.38%] [G loss: 1.115541]
310 [D loss: 0.735067, acc.: 57.81%] [G loss: 0.968792]
311 [D loss: 0.834283, acc.: 46.88%] [G loss: 0.907931]
312 [D loss: 0.742313, acc.: 50.00%] [G loss: 1.333000]
313 [D loss: 0.816630, acc.: 43.75%] [G loss: 1.070421]
314 [D loss: 0.790885, acc.: 48.44%] [G loss: 0.903339]
315 [D loss: 0.747627, acc.: 60.94%] [G loss: 1.160599]
316 [D loss: 0.809071, acc.: 43.75%] [G loss: 0.962097]
317 [D loss: 0.771598, acc.: 54.69%] [G loss: 1.146629]
318 [D loss: 0.788141, acc.: 50.00%] [G loss: 1.

...

3281 [D loss: 0.581646, acc.: 71.88%] [G loss: 1.133041]
3282 [D loss: 0.666459, acc.: 64.06%] [G loss: 0.950930]
3283 [D loss: 0.595757, acc.: 71.88%] [G loss: 1.028314]
3284 [D loss: 0.723440, acc.: 51.56%] [G loss: 0.970775]
3285 [D loss: 0.622235, acc.: 59.38%] [G loss: 0.932201]
3286 [D loss: 0.600567, acc.: 65.62%] [G loss: 1.347044]
3287 [D loss: 0.864513, acc.: 32.81%] [G loss: 0.884096]
3288 [D loss: 0.696416, acc.: 56.25%] [G loss: 0.779708]
3289 [D loss: 0.657120, acc.: 59.38%] [G loss: 0.718665]
3290 [D loss: 0.600777, acc.: 70.31%] [G loss: 1.143075]
3291 [D loss: 0.678061, acc.: 59.38%] [G loss: 0.930425]
3292 [D loss: 0.739326, acc.: 51.56%] [G loss: 0.971302]
3293 [D loss: 0.647972, acc.: 64.06%] [G loss: 1.077663]
3294 [D loss: 0.697181, acc.: 60.94%] [G loss: 0.998435]
3295 [D loss: 0.611388, acc.: 75.00%] [G loss: 1.030033]
3296 [D loss: 0.579648, acc.: 76.56%] [G loss: 1.026155]
3297 [D loss: 0.739773, acc.: 45.31%] [G loss: 0.989233]
3298 [D loss: 0.588156, acc.: 6

3431 [D loss: 0.523315, acc.: 78.12%] [G loss: 1.260464]
3432 [D loss: 0.656447, acc.: 60.94%] [G loss: 0.933447]
3433 [D loss: 0.615719, acc.: 67.19%] [G loss: 0.896562]
3434 [D loss: 0.576696, acc.: 65.62%] [G loss: 1.003533]
3435 [D loss: 0.675054, acc.: 60.94%] [G loss: 1.035536]
3436 [D loss: 0.773884, acc.: 48.44%] [G loss: 0.809830]
3437 [D loss: 0.628505, acc.: 64.06%] [G loss: 1.122651]
3438 [D loss: 0.654302, acc.: 62.50%] [G loss: 1.122406]
3439 [D loss: 0.698496, acc.: 54.69%] [G loss: 0.995335]
3440 [D loss: 0.863528, acc.: 50.00%] [G loss: 0.857665]
3441 [D loss: 0.693319, acc.: 67.19%] [G loss: 0.983908]
3442 [D loss: 0.679460, acc.: 60.94%] [G loss: 0.886585]
3443 [D loss: 0.575998, acc.: 75.00%] [G loss: 1.159524]
3444 [D loss: 0.728406, acc.: 56.25%] [G loss: 1.046170]
3445 [D loss: 0.704225, acc.: 59.38%] [G loss: 1.016583]
3446 [D loss: 0.631622, acc.: 60.94%] [G loss: 0.997578]
3447 [D loss: 0.678220, acc.: 51.56%] [G loss: 0.861582]
3448 [D loss: 0.631141, acc.: 7

3582 [D loss: 0.712015, acc.: 60.94%] [G loss: 1.001544]
3583 [D loss: 0.690575, acc.: 53.12%] [G loss: 0.972584]
3584 [D loss: 0.616414, acc.: 67.19%] [G loss: 1.019919]
3585 [D loss: 0.688521, acc.: 56.25%] [G loss: 1.141745]
3586 [D loss: 0.621453, acc.: 67.19%] [G loss: 1.044060]
3587 [D loss: 0.590054, acc.: 71.88%] [G loss: 1.035533]
3588 [D loss: 0.732763, acc.: 46.88%] [G loss: 0.949196]
3589 [D loss: 0.659780, acc.: 59.38%] [G loss: 1.041387]
3590 [D loss: 0.621811, acc.: 67.19%] [G loss: 0.767321]
3591 [D loss: 0.685405, acc.: 60.94%] [G loss: 1.043002]
3592 [D loss: 0.640523, acc.: 60.94%] [G loss: 1.130946]
3593 [D loss: 0.631350, acc.: 60.94%] [G loss: 1.045822]
3594 [D loss: 0.661053, acc.: 65.62%] [G loss: 1.089489]
3595 [D loss: 0.701529, acc.: 51.56%] [G loss: 0.924594]
3596 [D loss: 0.725932, acc.: 56.25%] [G loss: 1.083316]
3597 [D loss: 0.680071, acc.: 60.94%] [G loss: 1.023046]
3598 [D loss: 0.707220, acc.: 54.69%] [G loss: 0.973472]
3599 [D loss: 0.617380, acc.: 6

3731 [D loss: 0.687181, acc.: 59.38%] [G loss: 1.072177]
3732 [D loss: 0.623467, acc.: 71.88%] [G loss: 0.977529]
3733 [D loss: 0.621209, acc.: 68.75%] [G loss: 0.940654]
3734 [D loss: 0.606320, acc.: 68.75%] [G loss: 0.998372]
3735 [D loss: 0.742742, acc.: 48.44%] [G loss: 0.921664]
3736 [D loss: 0.574364, acc.: 79.69%] [G loss: 1.019472]
3737 [D loss: 0.633816, acc.: 68.75%] [G loss: 0.950609]
3738 [D loss: 0.644580, acc.: 62.50%] [G loss: 1.048878]
3739 [D loss: 0.683676, acc.: 56.25%] [G loss: 0.998437]
3740 [D loss: 0.627090, acc.: 68.75%] [G loss: 0.999820]
3741 [D loss: 0.641398, acc.: 62.50%] [G loss: 1.127885]
3742 [D loss: 0.613953, acc.: 65.62%] [G loss: 1.029637]
3743 [D loss: 0.734197, acc.: 53.12%] [G loss: 0.941869]
3744 [D loss: 0.556339, acc.: 78.12%] [G loss: 0.966776]
3745 [D loss: 0.668117, acc.: 57.81%] [G loss: 1.026652]
3746 [D loss: 0.574037, acc.: 75.00%] [G loss: 1.176855]
3747 [D loss: 0.646726, acc.: 64.06%] [G loss: 1.005916]
3748 [D loss: 0.668844, acc.: 5

3881 [D loss: 0.597481, acc.: 67.19%] [G loss: 0.901204]
3882 [D loss: 0.621680, acc.: 64.06%] [G loss: 1.173921]
3883 [D loss: 0.746689, acc.: 51.56%] [G loss: 0.953030]
3884 [D loss: 0.750128, acc.: 48.44%] [G loss: 0.960128]
3885 [D loss: 0.625991, acc.: 67.19%] [G loss: 1.001371]
3886 [D loss: 0.675782, acc.: 54.69%] [G loss: 0.834182]
3887 [D loss: 0.693185, acc.: 60.94%] [G loss: 1.083383]
3888 [D loss: 0.776803, acc.: 45.31%] [G loss: 1.051312]
3889 [D loss: 0.755464, acc.: 45.31%] [G loss: 1.047715]
3890 [D loss: 0.567655, acc.: 76.56%] [G loss: 1.029170]
3891 [D loss: 0.697136, acc.: 62.50%] [G loss: 1.053301]
3892 [D loss: 0.693433, acc.: 53.12%] [G loss: 1.075623]
3893 [D loss: 0.673034, acc.: 60.94%] [G loss: 1.005146]
3894 [D loss: 0.631585, acc.: 64.06%] [G loss: 1.031628]
3895 [D loss: 0.736793, acc.: 50.00%] [G loss: 0.777816]
3896 [D loss: 0.674177, acc.: 64.06%] [G loss: 0.850630]
3897 [D loss: 0.689153, acc.: 60.94%] [G loss: 0.803155]
3898 [D loss: 0.624370, acc.: 6